# SQL Analysis – Day 1 Quick Checks
Data: `data/processed/clean_data.csv`  

In [9]:
from pathlib import Path

PROJ_ROOT = Path.cwd()
while PROJ_ROOT != PROJ_ROOT.parent and not (PROJ_ROOT / "data").exists():
    PROJ_ROOT = PROJ_ROOT.parent

DATA_PATH = PROJ_ROOT / "data" / "processed" / "clean_data.csv"
DB_PATH = PROJ_ROOT / "data" / "processed" / "day1_analysis.sqlite"


In [10]:
df = pd.read_csv(DATA_PATH)
conn = sqlite3.connect(DB_PATH)
df.to_sql("events", conn, if_exists="replace", index=False)
print(f"Loaded {len(df):,} rows into {DB_PATH}")

Loaded 90,189 rows into c:\Users\umyana\Documents\mobile_game_analytics_pipeline\data\processed\day1_analysis.sqlite


In [11]:
def run_query(query, params=None):
    return pd.read_sql(query, conn, params=params)

In [13]:
query_channel_purchase = """
SELECT acquisition_channel,
       COUNT(*) AS users,
       AVG(purchase) AS purchase_rate
FROM events
GROUP BY acquisition_channel
ORDER BY purchase_rate DESC;
"""
run_query(query_channel_purchase)

,acquisition_channel,users,purchase_rate
0,Instagram,36281,0.056421
1,Facebook,26776,0.055535
2,TikTok,17979,0.055175
3,Organic,9153,0.055064


In [14]:
query_platform_metrics = """
SELECT platform,
       AVG(revenue) AS avg_revenue,
       AVG(ROI) AS avg_roi
FROM events
GROUP BY platform;
"""
run_query(query_platform_metrics)

,platform,avg_revenue,avg_roi
0,App Store,3.613153,1.684582
1,Google Play,0.412797,-0.700801


In [15]:
query_retention = """
SELECT acquisition_channel,
       AVG(CASE WHEN retention_1 THEN 1 ELSE 0 END) AS retention_d1,
       AVG(CASE WHEN retention_7 THEN 1 ELSE 0 END) AS retention_d7
FROM events
GROUP BY acquisition_channel
ORDER BY retention_d1 DESC;
"""
run_query(query_retention)

,acquisition_channel,retention_d1,retention_d7
0,Instagram,0.448637,0.187178
1,Organic,0.445865,0.187807
2,Facebook,0.442897,0.184157
3,TikTok,0.441404,0.185772


In [16]:
channel_x_platform_segmentation = """
SELECT acquisition_channel,
       platform,
       COUNT(*)                      AS users,
       AVG(purchase)                 AS purchase_rate,
       AVG(revenue)                  AS avg_revenue,
       AVG(ROI)                      AS avg_roi
FROM events
GROUP BY acquisition_channel, platform
ORDER BY acquisition_channel, platform;
"""
run_query(channel_x_platform_segmentation)

,acquisition_channel,platform,users,purchase_rate,avg_revenue,avg_roi
0,Facebook,App Store,6668,0.055189,3.590048,0.282160
1,Facebook,Google Play,20108,0.055649,0.410183,-0.853506
2,Instagram,App Store,8931,0.055649,3.474526,0.510663
3,Instagram,Google Play,27350,0.056673,0.430364,-0.812885
4,Organic,App Store,2363,0.052899,3.526489,10.754965
5,Organic,Google Play,6790,0.055817,0.403901,0.346338
6,TikTok,App Store,4537,0.061935,3.965133,1.332431
7,TikTok,Google Play,13442,0.052894,0.385459,-0.773259


In [17]:
session_buckets = """
WITH session_buckets AS (
    SELECT *,
           CASE
               WHEN session_count < 5   THEN '0-4'
               WHEN session_count < 15  THEN '5-14'
               WHEN session_count < 50  THEN '15-49'
               WHEN session_count < 100 THEN '50-99'
               ELSE '100+'
           END AS session_band
    FROM events
)
SELECT session_band,
       COUNT(*)              AS users,
       AVG(purchase)         AS purchase_rate,
       AVG(revenue)          AS avg_revenue
FROM session_buckets
GROUP BY session_band
ORDER BY MIN(session_count);

"""
run_query(session_buckets)

,session_band,users,purchase_rate,avg_revenue
0,0-4,21725,0.025178,0.514261
1,5-14,20601,0.038687,0.901026
2,15-49,24698,0.058102,1.327343
3,50-99,10649,0.079350,1.571637
4,100+,12516,0.112336,2.395426


In [18]:
cac_vs_revenue = """
SELECT acquisition_channel,
       AVG(CAC)              AS avg_cac,
       AVG(revenue)          AS avg_rev,
       AVG(revenue - CAC)    AS avg_margin,
       AVG(ROI)              AS avg_roi
FROM events
GROUP BY acquisition_channel
ORDER BY avg_margin DESC;

"""
run_query(cac_vs_revenue)

,acquisition_channel,avg_cac,avg_rev,avg_margin,avg_roi
0,Organic,0.3,1.210050,0.910050,3.033499
1,TikTok,1.7,1.288790,-0.411210,-0.241888
2,Instagram,2.3,1.179721,-1.120279,-0.487078
3,Facebook,2.8,1.202061,-1.597939,-0.570692


In [19]:
roi_outliers = """
SELECT platform,
       SUM(CASE WHEN ROI > 5 THEN 1 ELSE 0 END) AS roi_gt_5,
       COUNT(*)                                AS users,
       ROUND(100.0 * SUM(CASE WHEN ROI > 5 THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_roi_gt_5
FROM events
GROUP BY platform;

"""
run_query(roi_outliers)

,platform,roi_gt_5,users,pct_roi_gt_5
0,App Store,1176,22499,5.23
1,Google Play,536,67690,0.79


In [20]:
retention_and_purchase = """
SELECT acquisition_channel,
       AVG(purchase) AS purchase_rate,
       AVG(CASE WHEN retention_1 THEN 1 ELSE 0 END) AS retention_d1,
       AVG(CASE WHEN retention_7 THEN 1 ELSE 0 END) AS retention_d7
FROM events
GROUP BY acquisition_channel
ORDER BY retention_d1 DESC;
"""
run_query(retention_and_purchase)

,acquisition_channel,purchase_rate,retention_d1,retention_d7
0,Instagram,0.056421,0.448637,0.187178
1,Organic,0.055064,0.445865,0.187807
2,Facebook,0.055535,0.442897,0.184157
3,TikTok,0.055175,0.441404,0.185772


### SQL Checks Summary
- Channel-based purchase rates are very close to each other (around 5.5%); Instagram has a slight advantage at 5.64%.

- In terms of platform comparison, App Store users generate $3.61 revenue per person and a +1.68 ROI, while Google Play remains at a negative ROI with $0.41 revenue.

- Channel+platform segmentation shows that the TikTok x App Store combination delivers the highest purchase rate (6.19%) and positive ROI (1.33); TikTok x Google Play, however, produces a negative margin.

- As the number of sessions increases, conversion and revenue rise: 2.5% purchase in the 0-4 session band, while 11.2% purchase and $2.40 revenue/TBA are achieved in the 100+ band.

- CAC comparison: Organic stands out with low CAC (0.3) and a positive margin of 0.91; Instagram/Facebook channels generate a negative margin due to high CAC.

- ROI>5 observations are concentrated more in the App Store (5.2% of users); this rate is 0.79% in Google Play.

- When comparing retention and purchase, Instagram and Organic channels lead in D1/D7 retention; purchase rates also remain at similar levels.